In [ ]:
from bios_utils.problema import Problema
import pandas as pd

In [ ]:

problema = Problema(bios_input_file="data/0_model_template_1207.xlsm")

In [ ]:
# Obtener la lista de todas las exportaciones en un dataframe
despachos_df = problema.cargas_despachables_df[[
    'Empresa', 'Puerto', 'Operador', 'Ingrediente', 'Importacion']].drop_duplicates()

print(despachos_df.shape)

In [ ]:
despachos_df

In [ ]:
peridos_df = pd.DataFrame(problema.periodos, columns=['Fecha'])

In [ ]:
# Cruzar con la columna de periodos
despachos_df = pd.merge(left=despachos_df, right=peridos_df, how='cross')

print(despachos_df.shape)

In [ ]:
despachos_df = pd.merge(left=despachos_df,
                        right=problema.costos_almacenamiento_df.drop_duplicates(),
                        left_on=['Empresa', 'Puerto', 'Operador',
                                 'Ingrediente', 'Importacion', 'Fecha'],
                        right_on=['empresa', 'puerto', 'operador',
                                  'ingrediente', 'importacion', 'fecha_corte'],
                        how='left').drop(columns=['empresa', 'puerto', 'operador', 'ingrediente',
                                                  'importacion', 'fecha_corte']).rename(columns={'valor_kg': 'almacenamiento_kg'}).fillna(0.0)

print(despachos_df.shape)

In [ ]:
bodegaje_df = pd.merge(left=problema.tto_puerto_df.groupby(['Empresa', 'Puerto', 'Operador', 'Ingrediente', 'Importacion'])[['Fecha']].max().reset_index(),
                       right=problema.costo_portuario_bodegaje_df.rename(
                           columns={'valor_kg': 'bodegaje_kg'}),
                       left_on=['Puerto', 'Operador', 'Ingrediente'],
                       right_on=['puerto', 'operador', 'ingrediente'],
                       how='left').drop(columns=['puerto', 'operador', 'ingrediente'])
print(bodegaje_df.shape)

In [ ]:
despachos_df = pd.merge(left=despachos_df,
                        right=bodegaje_df,
                        left_on=['Empresa', 'Puerto', 'Operador',
                                 'Ingrediente', 'Importacion', 'Fecha'],
                        right_on=['Empresa', 'Puerto', 'Operador',
                                  'Ingrediente', 'Importacion', 'Fecha'],
                        how='left').fillna(0.0)
print(despachos_df.shape)

In [ ]:
fletes_df = problema.fletes_df.melt(id_vars=['puerto', 'operador', 'ingrediente'],
                                    value_vars=problema.fletes_df.drop(
                                        columns=['puerto', 'operador', 'ingrediente']).columns,
                                    var_name='planta',
                                    value_name='flete_kg')

In [ ]:
fletes_df['flete_camion'] = 34000*fletes_df['flete_kg']

In [ ]:
despachos_df = pd.merge(left=despachos_df,
                        right=fletes_df,
                        left_on=['Puerto', 'Operador', 'Ingrediente'],
                        right_on=['puerto', 'operador', 'ingrediente'],
                        how='left').drop(columns=['puerto', 'operador', 'ingrediente', 'flete_kg'])

print(despachos_df.shape)

In [ ]:
despachos_df = pd.merge(left=despachos_df,
                        right=problema.costo_portuario_directo_df.rename(
                            columns={'Directo': 'directo_kg'}),
                        left_on=['Operador', 'Puerto', 'Ingrediente'],
                        right_on=['Operador', 'Puerto', 'Ingrediente'],
                        how='left')

In [ ]:
despachos_df.shape

In [ ]:
despachos_df = pd.merge(left=despachos_df,
                        right=problema.transitos_a_puerto_df.reset_index().drop(
                            columns=['Inventario', 'valor_kg', 'Camiones']),
                        left_on=['Empresa', 'Puerto', 'Operador',
                                 'Ingrediente', 'Importacion', 'Fecha'],
                        right_on=['Empresa', 'Puerto', 'Operador',
                                  'Ingrediente', 'Importacion', 'Fecha'],
                        how='left')
despachos_df['Llegada'] = despachos_df['Llegada'].fillna(0.0)

In [ ]:
despachos_df['costo_almacenamiento_kg'] = despachos_df['bodegaje_kg'] + \
    despachos_df['almacenamiento_kg']

In [ ]:
despachos_df['costo_almacenamiento_camion'] = 34000 * \
    despachos_df['costo_almacenamiento_kg']

In [ ]:
# Extraer el vector de costos de almacenamiento
temp = despachos_df.pivot_table(index=['Empresa', 'Puerto', 'Operador', 'Ingrediente',
                                'Importacion'], columns='Fecha', values='costo_almacenamiento_camion', aggfunc='mean')

In [ ]:
from itertools import accumulate


def suma_acumulada_inversa(fila):
    fila_invertida = fila[::-1]
    suma_acumulada_invertida = list(accumulate(fila_invertida))
    return suma_acumulada_invertida[::-1]

In [ ]:
titulos = list(temp.columns)

In [ ]:
temp = temp[temp.columns].apply(suma_acumulada_inversa, axis=1).reset_index()

In [ ]:
for i in titulos:
    temp[i] = [temp[0][j][titulos.index(i)] for j in range(temp.shape[0])]

In [ ]:
temp = temp.melt(id_vars=['Empresa', 'Puerto', 'Operador', 'Ingrediente', 'Importacion'],
                 value_vars=titulos,
                 value_name='ahorro_almacenamiento_kg',
                 var_name='Fecha')

In [ ]:
despachos_df = pd.merge(left=despachos_df,
                        right=temp,
                        left_on=['Empresa', 'Puerto', 'Operador',
                                 'Ingrediente', 'Importacion', 'Fecha'],
                        right_on=['Empresa', 'Puerto', 'Operador',
                                  'Ingrediente', 'Importacion', 'Fecha'],
                        how='left')

In [ ]:
despachos_df[(despachos_df['Importacion'] == '5660SERVETTE')
             & (despachos_df['planta'] == 'envigado')].head()

In [ ]:
despachos_df['Costo_Camion'] = despachos_df.apply(
    lambda x: x['flete_camion'] + 34000*x['directo_kg'] if x['Llegada'] > 0 else x['flete_camion'], axis=1)

In [ ]:
despachos_df['Neto_Objetivo'] = despachos_df['Costo_Camion'] - \
    despachos_df['ahorro_almacenamiento_kg']

In [ ]:
despachos_df['Var_Name'] = despachos_df.apply(
    lambda x: f"{x['Empresa']}_{x['Puerto']}_{x['Operador']}_{x['Ingrediente']}_{x['Importacion']}_{x['Fecha']}", axis=1)

In [ ]:
despachos_df